# Working with Shap to do some vis on the D1 DAN model
Sean Browning

In [ ]:
# === Lib ==========
import os
import pickle as pkl
import shap

import numpy as np
import pandas as pd
import tensorflow.keras as keras
import tensorflow_addons as tfa
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

import tools.analysis as ta
import tools.preprocessing as tp
import tools.keras as tk
from scipy.sparse import lil_matrix
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import (Lasso, LogisticRegression, Ridge,
                                  SGDClassifier)


In [ ]:
# === GLOBALS ======
OUTCOME = "misa_pt"
DAY_ONE_ONLY = True
WEIGHTED_LOSS = False
TEST_SPLIT = 0.2
VAL_SPLIT = 0.1
RAND = 2021
TB_UPDATE_FREQ = 100
MOD_NAME = "dan"
TIME_SEQ = 225
BATCH_SIZE = 128
EPOCHS = 20

# === DIRS ================

pwd = "C:/Users/blues/work/premier_analysis/python"

# If no args are passed to overwrite these values, use repo structure to construct
data_dir = os.path.abspath(os.path.join(pwd, "..", "data", "data", ""))
output_dir = os.path.abspath(os.path.join(pwd, "..", "output", ""))

tensorboard_dir = os.path.abspath(
    os.path.join(data_dir, "..", "model_checkpoints"))
pkl_dir = os.path.join(output_dir, "pkl")
stats_dir = os.path.join(output_dir, "analysis")
probs_dir = os.path.join(stats_dir, "probs")

## Load in data

In [ ]:
# Data load
with open(os.path.join(pkl_dir, OUTCOME + "_trimmed_seqs.pkl"), "rb") as f:
    inputs = pkl.load(f)

with open(os.path.join(pkl_dir, "all_ftrs_dict.pkl"), "rb") as f:
    vocab = pkl.load(f)

with open(os.path.join(pkl_dir, "feature_lookup.pkl"), "rb") as f:
    all_feats = pkl.load(f)

with open(os.path.join(pkl_dir, "demog_dict.pkl"), "rb") as f:
    demog_lookup = pkl.load(f)
    demog_lookup = {k: v for v, k in demog_lookup.items()}

## Model-specific settings

In [ ]:
# Separating the inputs and labels
features = [t[0] for t in inputs]
demog = [t[1] for t in inputs]
labels = [t[2] for t in inputs]

# Counts to use for loops and stuff
n_patients = len(features)
n_features = np.max(list(vocab.keys()))
n_classes = len(np.unique(labels))
binary = n_classes <= 2

# Converting the labels to an array
y = np.array(labels, dtype=np.uint8)

# Optionally limiting the features to only those from the first day
# of the actual COVID visit
if DAY_ONE_ONLY:
    features = [l[-1] for l in features]
else:
    features = [tp.flatten(l) for l in features]

# Optionally mixing in the demographic features
new_demog = [[i + n_features for i in l] for l in demog]
features = [features[i] + new_demog[i] for i in range(n_patients)]
demog_vocab = {k + n_features: v for k, v in demog_lookup.items()}
vocab.update(demog_vocab)
n_features = np.max([np.max(l) for l in features])
all_feats.update({v: v for k, v in demog_lookup.items()})

# Converting the features to a sparse matrix
mat = lil_matrix((n_patients, n_features + 1))
for row, cols in enumerate(features):
    mat[row, cols] = 1

# Converting to csr because the internet said it would be faster
X = mat

## Splitting the data

In [ ]:
# Splitting the data
train, test = train_test_split(range(n_patients),
                                test_size=TEST_SPLIT,
                                stratify=y,
                                random_state=RAND)

train, val = train_test_split(train,
                                test_size=VAL_SPLIT,
                                stratify=y[train],
                                random_state=RAND)

## Model Training

In [ ]:
mod = RandomForestClassifier(n_estimators=500, n_jobs=-1)
mod.fit(X[train], y[train])

## Working with Shapley
BUG: DeepExplainer is failing with Embedding layers. Not sure a workaround other than to use a basic kernel.


First we have to take a sample of training data

In [ ]:
# This is a list of samples we'll visualize
shap_train_sample_idx = np.random.choice(train, 100, replace=False)
shap_test_sample_idx = np.random.choice(test, 100, replace=False)

## Create our explainer
Generating shapley values

In [ ]:
explain = shap.Explainer(mod, X[shap_train_sample_idx].toarray())
shap_vals = explain.shap_values(X[shap_test_sample_idx].toarray())

### Back-transforming integer sequences to text names for vis

In [ ]:
# Convert sequence
inv_vocab = {k: v for v, k in vocab.items()}
inv_vocab_df = pd.DataFrame.from_dict(inv_vocab, "index", columns=["idx"])
all_feats_df = pd.DataFrame.from_dict(all_feats, "index", columns = ["val_txt"])

vocab_df = inv_vocab_df.join(all_feats_df, how = "left").reset_index()
vocab_df.set_index("idx", inplace=True)

vocab_df["val_txt"] = np.where(vocab_df["val_txt"].isnull(), vocab_df["index"], vocab_df["val_txt"])

vocab_df = vocab_df.sort_index()

feat_names = vocab_df["val_txt"].tolist()
feat_names.insert(0, "OOV")

### Visualize

In [ ]:
shap.initjs()
expected_val = explain.expected_value
print(f"Explainer expected value: {expected_val}")

shap.decision_plot(
    expected_val[0],
    shap_vals[0],
    link="logit",
    feature_names = feat_names,
    ignore_warnings=True
)
